In [0]:
import pyspark.sql.functions as F

In [0]:
df_source = spark.table("d_dnst_020_silver.silver_03.`03_sellout_with_place_id`")

In [0]:
df_source.count()

In [0]:
(
    df_source
    .filter(F.col("unique_id").isin("1022551752"))
    .select("customer_name", "unique_id", "customer_number", "customer_address", "vat_number")
    .display()
)

In [0]:
df_matched_internal = spark.table("d_dnst_020_silver.silver_03.`031_sellout_customers_int_matched`")

In [0]:
df_matched_internal.count()

In [0]:
from pyspark.sql import Window
window_spec = Window.partitionBy("unique_id")

(
    df_matched_internal
    # .withColumn(
    # "match_count", 
    # F.count("*").over(window_spec)
    # )
    # .filter(
    # F.col("match_count") > 2
    # )
    .orderBy("unique_id")
    .filter(F.col("unique_id").isin('649495686', "525000170"))
    .display()
)

In [0]:
(
    df_matched_internal
    .filter(F.col("dim_sellout_cust_id").isNull())
    .count()
)

In [0]:
from pyspark.sql import functions as F

cols = [
    "int_customer_number_match",
    "int_vat_number_match",
    "int_place_id_match",
    "int_levenstein_name_address"
]

# Total rows
total_rows = df_matched_internal.count()

# Compute non-null counts for each column
percentages = []
for col in cols:
    non_null_count = df_matched_internal.filter(F.col(col).isNotNull()).count()
    percentages.append((col, (non_null_count / total_rows) * 100))

# Compute coalesce non-null percentage
coalesce_non_null_count = df_matched_internal.filter(
    F.coalesce(*[F.col(c) for c in cols]).isNotNull()
).count()
coalesce_percentage = (coalesce_non_null_count / total_rows) * 100

# Display results
for col, pct in percentages:
    print(f"{col}: {pct:.2f}% non-null")
print(f"Coalesce of all columns: {coalesce_percentage:.2f}% non-null")

In [0]:
(
    df_matched_internal
    .select(
        # "dim_sellout_cust_id",
        "customer_number", 
        "customer_name", 
        "customer_address", 
        "customer_postal_code",
        "vat_number", 
        # "place_id",
        # "int_customer_number_match", 
        "int_vat_number_match", 
        # "int_place_id_match", 
        "int_levenstein_name_address",
        "levenstein_score",
        # "unique_id",
        # "value_dict",
        # "match_details",
        "dim_sellout_cust_id"
        )
    # .filter(F.col("int_customer_number_match").isNotNull())
    .orderBy(F.col("customer_number"))
    # .filter(F.col("unique_id").isin("221998417", "1665322919", "1898281668", "1440708323", "53412208", "1601883039", "1818981059"))
    # .filter(F.col("dim_sellout_cust_id") == "1353385097")
    .orderBy(F.col("customer_name"))
    .display()
)

In [0]:
df_joined = (
    df_matched_internal.alias("l")
    .join(
        df_matched_internal.alias("r"),
        on = F.expr("l.int_levenstein_name_address == r.unique_id"),
        how = "inner"
    )
    .selectExpr(
        "l.*",
        "r.customer_name as matched_name",
        "r.unique_id as matched_unique_id",
        # "r.dim_sellout_cust_id as matched_dim_sellout_cust_id",
        "r.customer_postal_code as matched_customer_postal_code",
        "r.vat_number as matched_vat_number",
        "r.customer_address as matched_customer_address",
        "r.customer_number as matched_customer_number",
        "r.dim_sellout_cust_id as matched_dim_sellout_cust_id"
    )
)

In [0]:
(
    df_joined

    # .filter(F.col("unique_id").isin("1330801751", "1072653254"))
    # .filter(F.col("customer_postal_code") != F.col("matched_customer_postal_code"))
    .withColumn(
        "similarity_score", 1- (
        F.levenshtein(F.upper(F.col("customer_address")), F.upper(F.col("matched_customer_address"))) /
        F.greatest(F.length(F.col("customer_address")), F.length(F.col("matched_customer_address"))))
    )
    .select("unique_id","matched_unique_id","similarity_score", "customer_name", "matched_name","customer_address", "matched_customer_address","matched_customer_number", "customer_number", "value_dict","customer_postal_code", "matched_customer_postal_code", "matched_vat_number","vat_number","matched_dim_sellout_cust_id", "dim_sellout_cust_id")
    .orderBy(F.asc(F.col("similarity_score")))
    # .filter(F.col("dim_sellout_cust_id") != F.col("matched_dim_sellout_cust_id"))
    .display()
)

In [0]:
(
    df_joined
    .filter(F.col("dim_sellout_cust_id").isin('1003249906'))
    .select("unique_id", "customer_name","customer_address","matched_customer_number","place_id", "customer_number",  "dim_sellout_cust_id")
    .display()

)

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# Define the window partitioned by dim_sellout_cust_id
window_spec = Window.partitionBy("dim_sellout_cust_id")

# Add a column with the count per dim_sellout_cust_id
df_with_counts = df_matched_internal.withColumn(
    "cust_id_count",
    F.count("*").over(window_spec)
)

# Now select distinct dim_sellout_cust_id and count, then order by count descending
result_df = (
    df_with_counts
    .select("dim_sellout_cust_id", "cust_id_count")
    .distinct()
    .orderBy(F.col("cust_id_count").desc())
)

display(result_df)
